In [61]:
import math
import numpy as np

In [62]:
def ordenar_info(info):
    maps_dict = {}
    current_map = []
    current_map_key = None

    for line in content:
        if "map" in line:
            if current_map_key and current_map:
                maps_dict[current_map_key] = current_map
                current_map = []
            current_map_key = line.strip(':')
        elif "seeds" in line:
            seeds_line = line.split(':')
            if len(seeds_line) == 2:
                seeds_key = seeds_line[0].strip()
                seeds_values = list(map(int, seeds_line[1].split()))
                maps_dict[seeds_key] = seeds_values
        elif current_map_key:
            current_map.append(list(map(int, line.split())))

    # Asegurarse de agregar el último map
    if current_map_key and current_map:
        maps_dict[current_map_key] = current_map

    return maps_dict

In [68]:
def get_next_step(info_map, step_origen, print_debug=False):
    step_destino = None
    for info in info_map:
        destino, origen, rango = info
        rango_origen = [ 0, 0 + rango ]
        rango_destino = [ 0, 0 + rango ]

        if step_origen >= rango_origen[0] + origen and step_origen < rango_origen[1] + origen:
            delta_index = step_origen - origen
            step_destino = delta_index + destino
    if not step_destino:
        step_destino = step_origen

    return step_destino

In [87]:
# ---- PART ONE ---- #

with open('input_5.txt') as f:
    content = f.readlines()
content = [x.replace('\n', '') for x in content if x != '\n']

info = ordenar_info(content)

seeds = info['seeds']
locations = math.inf

for seed in seeds:
    step = seed
    for mapping in info.keys():
        if 'map' in mapping:
            step = get_next_step(info[mapping], step)
    locations = min(locations, step)

print('Result:', locations)



Result: 240320250


In [71]:
# ---- PART TWO ---- #

with open('input_example.txt') as f:
    content = f.readlines()
content = [x.replace('\n', '') for x in content if x != '\n']

info = ordenar_info(content)

rango = None
seeds = []
for index, seed in enumerate(info['seeds']):
    if index % 2 == 0:
        rango = seed
    else:
        seeds.append([rango, seed]) 

true_seeds = []
for seed in seeds:
    result_array = [value for value in range(seed[0], seed[0] + seed[1])]
    true_seeds.extend(result_array)

print(len(true_seeds))
info['seeds'] = true_seeds

# 1394379071 in 1m 49.6s

TypeError: 'int' object is not callable

In [34]:
locations = math.inf
total_seeds = len(info['seeds'])
for seed in info['seeds']:

    soil = get_next_step(info['seed-to-soil map'], seed)
    fertilizer = get_next_step(info['soil-to-fertilizer map'], soil)
    water = get_next_step(info['fertilizer-to-water map'], fertilizer)
    light = get_next_step(info['water-to-light map'], water)
    temperature = get_next_step(info['light-to-temperature map'], light)
    humidity = get_next_step(info['temperature-to-humidity map'], temperature)
    location = get_next_step(info['humidity-to-location map'], humidity)
    locations = min(locations, location)

    # Imprimir progreso cada 0.1%
    if (index + 1) % max(1, total_seeds // 1000) == 0:
        print(f"Progress: {((index + 1) / total_seeds) * 100:.1f}%")
    
print('Result:', locations)

In [85]:
info = ordenar_info(content)
seeds = info['seeds']
locations = math.inf

for seed in seeds:
    step = seed
    for mapping in info.keys():
        if 'map' in mapping:
            step = get_next_step(info[mapping], step)
    locations = min(locations, step)

print('Result:', locations)



Result: 240320250


## ROBADO 💩

In [27]:
import numpy as np

def get_next_step(info_map, step_origen, print_debug=False):
    info_map = np.array(info_map)
    
    destino = info_map[:, 0]
    origen = info_map[:, 1]
    rango = info_map[:, 2]
    
    rango_origen = origen + rango
    
    # Encuentra el índice del rango en el que se encuentra step_origen
    index_rango = np.where((step_origen >= origen) & (step_origen < rango_origen))[0]
    
    if len(index_rango) == 1:
        index_rango = index_rango[0]
        
        # Encuentra el valor en el rango destino usando la posición de step_origen en el rango origen
        posicion_en_rango = step_origen - origen[index_rango]
        step_destino = destino[index_rango] + posicion_en_rango
        
        if print_debug:
            print(f"Step origen: {step_origen}, Step destino: {step_destino}")
        
        return int(step_destino)
    else:
        # Si step_origen no está en ningún rango, devuelve step_origen sin cambios
        if print_debug:
            print(f"Step origen: {step_origen}, Step destino: {step_origen}")
        
        return int(step_origen)

# Ejemplo de uso
info_map_ejemplo = np.array([[50, 98, 2], [52, 50, 48]])
step_origen_ejemplo = 13

result = get_next_step(info_map_ejemplo, step_origen_ejemplo, print_debug=True)
print(f"Resultado: {result}")


Step origen: 13, Step destino: 13
Resultado: 13


In [33]:
import sys
from collections import defaultdict

with open('input_5.txt') as f:
    lines = [line.strip() for line in f.readlines()]

seeds_line = lines[0]
print(seeds_line)
seeds = [int(x) for x in seeds_line.split(':')[1].split(' ') if x]

mappings = []

new_mapping = False
mapping = None
for line in lines[1:]:
    if not line:
        new_mapping = True
    elif new_mapping:
        new_mapping = False
        mappings.append([])
    else:
        [dest, source, range] = [int(x) for x in line.split(' ')]
        mappings[-1].append((source, dest, range))

for mapping in mappings:
    mapping.sort()


seed_start = None
ranges = []
for i, seed_number in enumerate(seeds):
    if i % 2 == 0:
        seed_start = seed_number
    else:
        ranges.append((seed_start, seed_start + seed_number - 1))


ranges.sort()
print(len(seeds))

for iteration, mapping in enumerate(mappings):
    i = 0
    j = 0
    new_ranges = []

    range = None
    while ranges or range:
        if not range:
            range = ranges.pop(0)

        range_start = range[0]
        range_end = range[1]

        if j >= len(mapping):
            new_ranges.append((range_start, range_end))
            range = None
            i += 1
            continue

        mapping_start = mapping[j][0]
        mapping_end = mapping[j][0] + mapping[j][2] - 1

        mapping_shift = mapping[j][1] - mapping[j][0]

        if range_start < mapping_start and range_end < mapping_start:
            new_ranges.append((range_start, range_end))
            range = None
            i += 1
        elif range_start >= mapping_start and range_end <= mapping_end:
            new_ranges.append((range_start + mapping_shift, range_end + mapping_shift))
            range = None
            i += 1
        elif range_start <= mapping_start and range_end <= mapping_end:
            if range_start < mapping_start:
                new_ranges.append((range_start, mapping_start - 1))
            new_ranges.append((mapping_start + mapping_shift, range_end + mapping_shift))
            range = None
            i += 1
        elif range_start <= mapping_start and range_end >= mapping_end:
            if range_start < mapping_start:
                new_ranges.append((range_start, mapping_start - 1))
            new_ranges.append((mapping_start + mapping_shift, mapping_end + mapping_shift))
            if range_end > mapping_end:
                ranges = [(mapping_end + 1, range_end)] + ranges
            range = None
        elif range_start <= mapping_end and range_end > mapping_end:
            new_ranges.append((range_start + mapping_shift, mapping_end + mapping_shift))
            ranges = [(mapping_end + 1, range_end)] + ranges
            range = None
        else:
            j += 1

    ranges = new_ranges
    ranges.sort()

print(min(ranges)[0])

20
28580589


In [51]:
# algo = []
algo.append([])
algo[-1].append(1)

for mapping in algo:
    mapping.sort()

algo

[[1], [1], [2], [2], [2], [4], [1]]

In [54]:
import sys
from collections import defaultdict

with open('input_5.txt') as f:
    lines = [line.strip() for line in f.readlines()]

seeds_line = lines[0]
print(seeds_line)
seeds = [int(x) for x in seeds_line.split(':')[1].split(' ') if x]

mappings = []

new_mapping = False
mapping = None
for line in lines[1:]:
    if not line:
        new_mapping = True
    elif new_mapping:
        new_mapping = False
        mappings.append([])
    else:
        [dest, source, range] = [int(x) for x in line.split(' ')]
        mappings[-1].append((source, dest, range))

for mapping in mappings:
    mapping.sort()


seed_start = None
ranges = []
for i, seed_number in enumerate(seeds):
    if i % 2 == 0:
        seed_start = seed_number
    else:
        ranges.append((seed_start, seed_start + seed_number - 1))


ranges.sort()
print(len(seeds))

seeds: 3429320627 235304036 1147330745 114559245 1684000747 468955901 677937579 96599505 1436970021 26560102 3886049334 159534901 936845926 25265009 3247146679 95841652 3696363517 45808572 2319065313 125950148
20
